<a href="https://colab.research.google.com/github/ollpp/data_modeling/blob/master/NLP_Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git remote add origin https://github.com/ollpp/NLP_Assign.git
!git branch -M main
!git push -u origin main


error: refname refs/heads/master not found
fatal: Branch rename failed
error: src refspec main does not match any.
error: failed to push some refs to 'https://github.com/ollpp/NLP_Assign.git'


In [ ]:
!ls -lrt

total 12
drwxr-xr-x 1 root root 4096 Jul 16 13:20 sample_data
drwx------ 5 root root 4096 Jul 25 11:05 gdrive
drwxr-xr-x 3 root root 4096 Jul 25 11:16 NLP_Assign


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 448 kB 55.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
import re

import konlpy
from konlpy.tag import Okt

from pprint import pprint
import nltk

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

import json
import os

okt = Okt()

# 리뷰 데이터 
# g_rv : 게임 리뷰 데이터
# m_rv : 영화 레뷰 데이터(txt파일이기 때문에 \t를 기준으로 나눔)

# g_rv = pd.read_csv('/content/gdrive/My Drive/text_/Nexon_Assignment/game_rv.csv')
# m_rv = pd.read_csv('/content/gdrive/My Drive/text_/Nexon_Assignment/movie_rv.txt', delimiter='\t')

In [ ]:
# 게임 리뷰 데이터 형태와 차수 조회
g_rv.head()

,Unnamed: 0,Unnamed: 0.1,txt,label
0,0,0,겁나 튕깁니다 혹시 해결법아시는분 계신가요,NaN
1,1,1,원기야 더시드 좀 바꿔라,NaN
2,2,2,1채널 헤네시스 '아잉' 점령완료,NaN
3,3,3,몬스터 파크 신비의숲 아르카나 나와 주세요,NaN
4,4,4,팬텀 100랩 퀘스트,NaN


In [ ]:
# 리뷰 내용과 label만 남긴다
# 게임 리뷰를 0으로 labeling 한다.

del g_rv['Unnamed: 0']
del g_rv['Unnamed: 0.1']
g_rv['label']=0

In [ ]:
# 영화 리뷰 데이터 형태와 차수 조회
m_rv.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
# 50000개에서 20000개로 축소
# 리뷰 내용과 label만 남긴다.
# 영화 리뷰를 1으로 labeling 한다.

m_rv = m_rv[0:20000]
del m_rv['id']
m_rv.rename(columns={'document':'txt'}, inplace=True)
m_rv['label']=1

In [ ]:
# 영화 리뷰 데이터 형태와 차수 조회
m_rv.head()

,txt,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,1
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,1
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,1
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,1


In [ ]:
# 게임 및 영화 데이터를 병합하여 준다.
# index를 순서대로 재배열 하기 위해 ignore_index=True를 사용한다.

rv_train = pd.concat([g_rv, m_rv], ignore_index=True)

In [ ]:
rv_train.head()

,txt,label
0,겁나 튕깁니다 혹시 해결법아시는분 계신가요,0
1,원기야 더시드 좀 바꿔라,0
2,1채널 헤네시스 '아잉' 점령완료,0
3,몬스터 파크 신비의숲 아르카나 나와 주세요,0
4,팬텀 100랩 퀘스트,0


## **DateSet을 test data로 호출**

In [ ]:
rv_test = pd.read_csv('/content/gdrive/My Drive/text_/Nexon_Assignment/dataset.csv')

In [ ]:
rv_test.head()

,index,txt
0,0,엘리시움 배리 개쩜;;
1,1,◑업ㄷㅔㅇㅣ트㈍rㄹr고 ㄱㅔ임에서뜨는데 없뎃할ㄹㅏ고해도◑업뎃ㅇㅏ 안ㄸㅓ요
2,2,요샌 네이버 평점도 믿을게 못되는군. 행동거지가 예측이될정도니 한심하고. 온갖 바보...
3,3,갑자기 0x10030008 이라고뜨고 원격호스트종료 나오며 로그인이되질 않습니다. ...
4,4,공포 코미디 영화 보다가 웃기는 처음이다 이 새벽에


In [ ]:
# train 데이터와 형태가 동일하게 dataframe 변경
# index column 삭제
# label column 추가
# label은 Nan값으로 준다.

rv_test = pd.DataFrame({'txt':rv_test['txt'], 'label':None})

In [ ]:
rv_test.head()

,txt,label
0,엘리시움 배리 개쩜;;,None
1,◑업ㄷㅔㅇㅣ트㈍rㄹr고 ㄱㅔ임에서뜨는데 없뎃할ㄹㅏ고해도◑업뎃ㅇㅏ 안ㄸㅓ요,None
2,요샌 네이버 평점도 믿을게 못되는군. 행동거지가 예측이될정도니 한심하고. 온갖 바보...,None
3,갑자기 0x10030008 이라고뜨고 원격호스트종료 나오며 로그인이되질 않습니다. ...,None
4,공포 코미디 영화 보다가 웃기는 처음이다 이 새벽에,None


## konlpy를 사용한 전처리

In [ ]:
# 정규표현식을 사용해 한글만 남기기

def regur(dataset):
  for i in range(len(dataset)):
    if type(dataset['txt'][i])==str: 
      convert = re.compile('[가-힣]+').findall(dataset['txt'][i])
      dataset['txt'][i] = ' '.join(convert)

regur(rv_train)
regur(rv_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
rv_train.head()

,txt,label
0,겁나 튕깁니다 혹시 해결법아시는분 계신가요,0
1,원기야 더시드 좀 바꿔라,0
2,채널 헤네시스 아잉 점령완료,0
3,몬스터 파크 신비의숲 아르카나 나와 주세요,0
4,팬텀 랩 퀘스트,0


In [ ]:
# konlpy에 적용하기 위해 dataframe에서 list형식으로 바꾸어준다.

train_list = rv_train.values.tolist()
test_list = rv_test.values.tolist()

In [ ]:
# list로 변환한 파일 확인

print(len(train_list[0]))
print(len(test_list[0]))
print(len(train_list))
print(len(test_list))

2
2
40000
10000


In [ ]:
# 결측값 처리
def del_none(dataset):
  for i in range(len(dataset)):
    if type(dataset[i][0])!=str:
      dataset[i][0]=''

del_none(train_list)
del_none(test_list)

In [ ]:
# konlpy의 okt 라이브러리를 통해 정규화 메소드 정의

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [ ]:
train_docu = [(tokenize(row[0]), row[1]) for row in train_list]
test_docu = [(tokenize(row[0]), row[1]) for row in test_list]

with open('/content/gdrive/My Drive/train.json', 'w', encoding='utf-8') as make_file:
  json.dump(train_docu, make_file, ensure_ascii=False, indent='\t')
# with open('test.json', 'w', encoding='utf-8') as make_file:
#   json.dump(test_docu, make_file, ensure_ascii=False, indent='\t')




In [ ]:
tokens = [t for d in train_docu for t in d[0]]

In [ ]:
train_list[0:10]

[['겁나 튕깁니다 혹시 해결법아시는분 계신가요', 0],
 ['원기야 더시드 좀 바꿔라', 0],
 ['채널 헤네시스 아잉 점령완료', 0],
 ['몬스터 파크 신비의숲 아르카나 나와 주세요', 0],
 ['팬텀 랩 퀘스트', 0],
 ['스틸좀하지마세용 송파구길마님', 0],
 ['모험가 직업의눈물', 0],
 ['이겜은 신기한게', 0],
 ['민사하시면 안되요', 0],
 ['도움말때문에 이상해졌어요', 0]]

In [ ]:
# nltk.text(tokens)

print(len(set(tokens)))

19417


In [ ]:
text = nltk.Text(tokens, name='리뷰데이터')

pprint(text.vocab().most_common(10))

[('하다/Verb', 9036),
 ('이/Josa', 7655),
 ('영화/Noun', 6970),
 ('에/Josa', 6092),
 ('보다/Verb', 5887),
 ('의/Josa', 4225),
 ('들/Suffix', 4050),
 ('가/Josa', 3915),
 ('을/Josa', 3664),
 ('도/Josa', 2908)]


In [ ]:
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docu]
test_x = [term_frequency(d) for d, _ in test_docu]
train_y = [c for _, c in train_docu]
test_y = [c for _, c in test_docu]

In [ ]:
len(test_y)

10000

In [ ]:
type(x_train[0])

numpy.ndarray

In [ ]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [ ]:
x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
y_test

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

Epoch 1/10
79/79 [==============================] - 4s 41ms/step - loss: 0.3944 - binary_accuracy: 0.8777
Epoch 2/10
79/79 [==============================] - 3s 40ms/step - loss: 0.0749 - binary_accuracy: 0.9695
Epoch 3/10
79/79 [==============================] - 9s 120ms/step - loss: 0.0594 - binary_accuracy: 0.9748
Epoch 4/10
79/79 [==============================] - 3s 41ms/step - loss: 0.0470 - binary_accuracy: 0.9795
Epoch 5/10
79/79 [==============================] - 3s 41ms/step - loss: 0.0408 - binary_accuracy: 0.9816
Epoch 6/10
79/79 [==============================] - 3s 41ms/step - loss: 0.0398 - binary_accuracy: 0.9822
Epoch 7/10
79/79 [==============================] - 3s 41ms/step - loss: 0.0344 - binary_accuracy: 0.9855
Epoch 8/10
79/79 [==============================] - 3s 41ms/step - loss: 0.0325 - binary_accuracy: 0.9850
Epoch 9/10
79/79 [==============================] - 3s 41ms/step - loss: 0.0315 - binary_accuracy: 0.9866
Epoch 10/10
313/313 [========================

In [ ]:
result_label=[[1,2],[2,3],[3,4],[4,5],[4,1]]
after = []


for i in range(len(result_label)):
  if (i>2):
    result_label[i][1] = 100
  else:
    result_label[i][1] = 1000
  

result_label

[[1, 1000], [2, 1000], [3, 1000], [4, 100], [4, 100]]

In [ ]:
result_label[i][1]

1

In [ ]:
len(rv_test['txt'])

10000

In [ ]:
def predict_pos_neg(review):    # rv_test
  result_label=[]
  for i in range(len(review['txt'])):
      token = tokenize(review['txt'][i])
      tf = term_frequency(token)
      data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
      score = float(model.predict(data))
      if(score > 0.5):
          # print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
          result_label.append(1)
      else:
          # print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))
          result_label.append(0)
  return result_label

In [ ]:
result = predict_pos_neg(rv_test)


In [ ]:
rv_test['label'][1]

In [ ]:
aa1 = train_list[0:10]


aa1[0][1]

0

In [ ]:
def tt(a, b, c=0):
  return a, b, c

In [ ]:
(rv_test)

In [ ]:
a1, a2, a3 = tt(1, 2, model)
type(a3)

tensorflow.python.keras.engine.sequential.Sequential

In [ ]:
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")

[배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.]는 100.00% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [ ]:
predict_pos_neg(rv_test['txt'][1539])

[실관 대한 조 가 진 이정도밖 는 이루 지 않늕다 진 능하 나 한일인 빡침 득한 영화 원장이 클 나 숨막히게 증 고 엔딩은 증나늖 영화 라리 안 보면 좋았을거 갇음]는 100.00% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [ ]:
predict_pos_neg("이케니아")

[이케니아]는 57.94% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [ ]:
rv_test.shape

(10000, 2)

In [ ]:
predict_pos_neg("스카니아")

[스카니아]는 99.99% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('/content/gdrive/My Drive/first_Model')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/first_Model/assets


In [ ]:
!mkdir -p model_saved